<a href="https://colab.research.google.com/github/Wesley-Janson/transformers_for_human_vs_ai_text_identification/blob/main/BoW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# importing the libraries
import pandas as pd
import numpy as np

# for reading and displaying images
import matplotlib.pyplot as plt

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# PyTorch libraries and modules
import torch
import torch.nn.functional as F
from torch import nn
from torch.nn import Linear, CrossEntropyLoss
from torch.optim import Adam
from torchtext.data.utils import get_tokenizer
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torchtext.vocab import build_vocab_from_iterator


In [11]:
PATH = 'Users/spavlekovsky/advanced ml/GPT-data-smaller.csv'

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
PATH = "gdrive/MyDrive/GPT-data-smaller.csv"

raw_data = pd.read_csv(PATH)

Mounted at /content/gdrive


In [3]:
raw_data = raw_data.iloc[1:100000]

In [4]:
raw_data = raw_data.loc[(raw_data['generated_intro_len'] >= 34) & (raw_data['wiki_intro_len'] >= 34)]
data = pd.concat([
    pd.DataFrame({'intro': raw_data['generated_intro'], 'label': 1}),
    pd.DataFrame({'intro': raw_data['wiki_intro'], 'label': 0})]
).reset_index().drop('index', axis=1)
data['intro'] = data['intro'].apply(lambda s: s.split(' ', 7)[-1])
text = data['intro'].values
labels = data['label'].values
print(data)

                                                    intro  label
0       animate or inanimate, have a spirit or "etiäin...      1
1       function theorem states that for every real-va...      1
2       and illustrated by Maki Fujii. The series foll...      1
3       29, 1973) is an American former professional b...      1
4       groups inhabiting the Maluku Islands. The term...      1
...                                                   ...    ...
195723  that airs on the CBBC Channel. Hero Squad foll...      0
195724  author Stephen King, writing under the pseudon...      0
195725  listed, early Edwardian estate towards the nor...      0
195726  start point, a series of control points, and a...      0
195727  terrace in Humboldt County, California, that t...      0

[195728 rows x 2 columns]


In [5]:
train_x, val_x, train_y, val_y = train_test_split(text, labels, test_size = 0.2)
val_x, test_x, val_y, test_y = train_test_split(val_x, val_y, test_size = 0.5)

tokenizer = get_tokenizer("basic_english")

In [6]:

MIN_VOCAB_FREQ = 200

In [7]:
train_iter = iter(train_x)

def yield_tokens(train_iter):
    for text in train_iter:
        yield tokenizer(text)
        
vocab = build_vocab_from_iterator(
    yield_tokens(train_iter), specials=["<unk>"], min_freq=MIN_VOCAB_FREQ)
vocab.set_default_index(vocab.lookup_indices(["<unk>"])[0])
len(vocab)

8670

In [8]:
print(train_x.shape)
print(train_y.shape)
print(val_x.shape)
print(val_y.shape)
print(test_x.shape)
print(test_y.shape)

(156582,)
(156582,)
(19573,)
(19573,)
(19573,)
(19573,)


In [9]:
train_both = list(pd.DataFrame((train_x, train_y)).T.itertuples(index=False, name=None))
val_both = list(pd.DataFrame((val_x, val_y)).T.itertuples(index=False, name=None))
test_both = list(pd.DataFrame((test_x, test_y)).T.itertuples(index=False, name=None))


In [10]:
from random import randint

def collate_into_bow(batch):
    batch_labels = torch.tensor([s[1] for s in batch])
    batch_tokens = torch.zeros([len(batch), len(vocab)])
    for i, s in enumerate(batch):
        all_tokens = tokenizer(s[0])
        if len(all_tokens) <= 27:
          continue
        start_index = randint(0, len(all_tokens)-27)
        sample_tokens = all_tokens[start_index:start_index+27]
        sample_dict = {}
        for t in sample_tokens:
            if t in sample_dict:
                sample_dict[t] += 1
            else:
                sample_dict[t] = 1
        for t in sample_dict:
            idx = vocab.lookup_indices([t])[0]
            batch_tokens[i, idx] = sample_dict[t]/27
    return batch_labels, batch_tokens

In [11]:
train_loader = DataLoader(train_both, batch_size=10, shuffle=False, collate_fn=collate_into_bow)
val_loader = DataLoader(train_both, batch_size=10, shuffle=False, collate_fn=collate_into_bow)
test_loader = DataLoader(test_both, batch_size=10, shuffle=False, collate_fn=collate_into_bow)

In [12]:
class BoWClassifier(nn.Module):
    def __init__(self, vocab_size, num_labels=2):
        super(BoWClassifier, self).__init__()
        self.linear = nn.Linear(vocab_size, num_labels)
        
    def forward(self, bow_vec):
        return F.log_softmax(self.linear(bow_vec), dim=1)

In [13]:

loss_function = torch.nn.CrossEntropyLoss()

def train_an_epoch(dataloader, optimizer):
    model.train() # Sets the module in training mode.
    log_interval = 1000

    for idx, (label, text) in enumerate(dataloader):
        model.zero_grad()
        log_probs = model(text)
        loss = loss_function(log_probs, label)
        loss.backward()
        optimizer.step()

In [14]:
def get_accuracy(dataloader):
    model.eval()
    batch_accuracies = []
    with torch.no_grad():
        for idx, (label, text) in enumerate(dataloader):
            log_probs = model(text)
            matches = torch.eq(log_probs.argmax(1), label.T)
            batch_accuracies.append(matches.sum()/len(label))
    return torch.mean(torch.stack(batch_accuracies)).item()

In [15]:
vocab_size = len(vocab)
model = BoWClassifier(vocab_size, 2)

EPOCHS = 15 # epoch
optimizer = torch.optim.Adam(model.parameters(), lr=3)

best_model = None
accuracies = []
for epoch in range(1, EPOCHS + 1):
    train_an_epoch(train_loader, optimizer)
    accuracy = get_accuracy(val_loader)
    accuracies.append(accuracy)
    print()
    print(f'After epoch {epoch} the validation accuracy is {accuracy:.3f}')
    print()
    if accuracies[-1] == max(accuracies):
        best_model = BoWClassifier(vocab_size, 2)
        best_model.load_state_dict(model.state_dict())
    
plt.plot(range(1, EPOCHS+1), accuracies)

tensor([[ 0.0000e+00, -5.1581e+01],
        [ 0.0000e+00, -1.7512e+01],
        [-1.2011e+01, -6.0797e-06],
        [ 0.0000e+00, -6.3920e+01],
        [-1.3901e+01, -9.5367e-07],
        [ 0.0000e+00, -2.0894e+01],
        [-6.1298e+01,  0.0000e+00],
        [-1.8200e+01,  0.0000e+00],
        [-1.1271e+01, -1.2755e-05],
        [ 0.0000e+00, -1.7868e+01]])
tensor([[ 0.0000e+00, -4.9693e+01],
        [ 0.0000e+00, -2.9493e+01],
        [-3.2638e+01,  0.0000e+00],
        [-3.1427e+01,  0.0000e+00],
        [-9.0473e+00, -1.1765e-04],
        [-1.1921e-06, -1.3635e+01],
        [ 0.0000e+00, -3.4217e+01],
        [ 0.0000e+00, -3.2158e+01],
        [-2.5942e+01,  0.0000e+00],
        [ 0.0000e+00, -5.4005e+01]])
tensor([[-1.2235e-02, -4.4095e+00],
        [-4.6492e-06, -1.2292e+01],
        [ 0.0000e+00, -2.8611e+01],
        [-3.9473e+01,  0.0000e+00],
        [-3.5956e+01,  0.0000e+00],
        [-1.1921e-07, -1.5640e+01],
        [-6.0862e+00, -2.2766e-03],
        [ 0.0000e+00, -2.4

<ipython-input-14-977a8f420bcd>:8: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3571.)
  matches = torch.eq(log_probs.argmax(1), label.T)


Streaming output truncated to the last 5000 lines.
tensor([[-8.7853e-01, -5.3682e-01],
        [ 0.0000e+00, -7.4368e+01],
        [-3.4610e+01,  0.0000e+00],
        [-1.1676e-03, -6.7534e+00],
        [-2.2944e+01,  0.0000e+00],
        [-2.1760e-01, -1.6319e+00],
        [ 0.0000e+00, -3.2258e+01],
        [-1.2353e+01, -4.2915e-06],
        [ 0.0000e+00, -3.4370e+01],
        [-4.3887e-02, -3.1480e+00]])
tensor([[ 0.0000e+00, -3.8123e+01],
        [-8.3751e+00, -2.3052e-04],
        [-5.0142e+01,  0.0000e+00],
        [ 0.0000e+00, -5.3439e+01],
        [ 0.0000e+00, -4.0553e+01],
        [-2.2650e-05, -1.0695e+01],
        [-1.5299e+01, -2.3842e-07],
        [-2.1585e+01,  0.0000e+00],
        [-4.2090e-01, -1.0684e+00],
        [ 0.0000e+00, -2.2318e+01]])
tensor([[ 0.0000e+00, -8.3126e+01],
        [ 0.0000e+00, -2.9349e+01],
        [-5.6990e+01,  0.0000e+00],
        [ 0.0000e+00, -1.6723e+01],
        [-4.4330e+00, -1.1950e-02],
        [-2.1583e+01,  0.0000e+00],
        [ 0

KeyboardInterrupt: ignored